In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from imports import *
from datetime import datetime, timedelta
import staircase as sc
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


In [ ]:
df = pd.read_csv(r"..//data/vessel_queue.csv", parse_dates=['enter', 'leave'], dayfirst=True)
dfh = pd.read_csv(r"../data/hotel_stays.csv", parse_dates=['check_in', 'check_out'], dayfirst=True)
#df.sort_values(by=['leave'],inplace=True)
#queue = sc.Stairs(use_dates=True).layer(df.enter, df.leave)
dfh.head()

In [ ]:
#%%timeit -r 3 -n 3
vsteps = Steps.read_dataframe(df,'enter','leave')

In [ ]:
%%timeit -r 3 -n 3
hotel_stays = Steps.read_dataframe(dfh,'check_in', 'check_out')

In [ ]:
%%timeit -r 3 -n 3
queue = sc.Stairs(use_dates=True).layer(dfh.check_in, dfh.check_out)

In [ ]:
from numpy import datetime64

def monkeypatch_method(cls):
    def decorator(func):
        setattr(cls,func.__name__,func)
        return func
    return decorator

#@monkeypatch_method(datetime64)
#def timestamp(self):
#    return (self - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')


def NaT_None(data):
    if pd.isna(data):
        return None
    else:
        return data

def create_step(d):
    return Step(use_datetime=True,data=d)


# st = Steps(True)
df2 = df.iloc[0:20]
# df2np = df2[['enter','leave']].to_numpy()
# df2npn = np.where(df2np is datetime64('NaT'),None,df2np)

# sss = np.apply_along_axis(create_step, 1, df2npn)
# sss
#dfc = df2[['enter','leave']].replace(np.datetime64('NaT'),None)
ss = df2.apply(lambda x: Step(use_datetime=True,data=(x['enter'],x['leave'])),axis=1)
#s = Step(data=np.array([None,100]))
print(ss)

In [ ]:
%%timeit -r 3 -n 3
ss = dfh.apply(lambda x: Step(use_datetime=True,data=(x['check_in'],x['check_out'])),axis=1)
#st = Steps(use_datetime=True).add(ss)

In [ ]:
%%timeit -r 3 -n 3
ss = dfh.apply(lambda x: Step(use_datetime=True,data=(NaT_None(x['check_in']),NaT_None(x['check_out']))),axis=1)
#st = Steps(use_datetime=True).add(ss)

In [ ]:
vsteps._step_np[0]

In [ ]:
step_dict = vsteps.to_dict()
print(step_dict.keys()[0:3],step_dict.values()[0:3])

In [ ]:
asteps = Steps.read_array(df.head(150).enter, df.head(150).leave,use_datetime=True)
dsteps = Steps.read_dict(step_dict)
step_dict.values()[0]

In [ ]:
df_vsteps = vsteps.to_dataframe() #
df_vsteps.head()

In [ ]:
vsteps_df = Steps.read_dataframe(df_vsteps,'start','end')

In [ ]:
#ax = vsteps_df.plot(size=(32,6))
ax = vsteps.plot(color='g')
asteps.plot(ax=ax,color='blue')
dsteps.plot(ax=ax,color='r')
#dsteps.normalise().plot(ax=ax,color='orange')

In [ ]:
d = pd.Timestamp(2020,1,20)
ax = vsteps_df.clip(lbound=d).plot(size=(32,6))
vsteps.clip(lbound=d).plot(ax=ax,color='g')
#asteps.clip(lbound=d).plot(ax=ax,color='blue')
dsteps.clip(lbound=d).plot(ax=ax,color='red')


In [ ]:
dsteps(pd.Timestamp(2020,1,1))

In [ ]:
dfdf = vsteps_df.clip(lbound=pd.Timestamp(2020,1,10),ubound=pd.Timestamp(2020,1,20)) #.to_dataframe(mode='aggregate')
vstep_clipped = vsteps.clip(lbound=pd.Timestamp(2020,1,10),ubound=pd.Timestamp(2020,1,20))
#vstep_clipped.steps()